In [1]:
import requests
import pandas as pd
import numpy as np

In [3]:
yummly_url1 = 'http://api.yummly.com/v1/api/recipes?_app_id=ab3c96d5&_app_key=57476e58107ffca18d1585f2b463d78e&q=pasta'

In [4]:
resp1 = requests.get(yummly_url1)

In [11]:
data1 = resp1.json()

In [18]:
data1.keys()

dict_keys(['criteria', 'matches', 'facetCounts', 'totalMatchCount', 'attribution'])

In [47]:
data1['matches'][3]

{'attributes': {'course': ['Main Dishes']},
 'flavors': {'bitter': 0.16666666666666666,
  'meaty': 0.6666666666666666,
  'piquant': 0.0,
  'salty': 0.16666666666666666,
  'sour': 0.8333333333333334,
  'sweet': 0.16666666666666666},
 'id': 'Lemon-Chicken-Pasta-2236012',
 'imageUrlsBySize': {'90': 'https://lh3.googleusercontent.com/ph6abYxDkKlr2GjUltVjS-0FAPfsD79PLxzTO3Fm9yaS6cN2FaSbVRkHRgd9puG8D_OHaObkUoo98D2wCpp40Q=s90-c'},
 'ingredients': ['linguine',
  'butter',
  'chicken',
  'chopped parsley',
  'lemon juice',
  'grated parmesan cheese',
  'lemon slices'],
 'rating': 4,
 'recipeName': 'Lemon Chicken Pasta',
 'smallImageUrls': ['https://lh3.googleusercontent.com/6eEhhDviHgt0rOj1sRMleu91iMs2FK9tqOyJbm5sXfy4Cpok5OjONIQ41RWzgWRFRDclvCZyeHoLtkXVRC5bIw=s90'],
 'sourceDisplayName': 'Framed Cooks',
 'totalTimeInSeconds': 1800}

In [21]:
data1['criteria']

{'allowedIngredient': None, 'excludedIngredient': None, 'q': 'pasta'}

In [22]:
data1['facetCounts']

{}

In [23]:
data1['totalMatchCount']

101657

In [42]:
params = {'maxResult' : 10, 'start' : 1}
resp1_b = requests.get(yummly_url1, params=params)

In [43]:
data1_b = resp1_b.json()

In [247]:
data1_b['matches'][5]

{'attributes': {'course': ['Main Dishes']},
 'flavors': None,
 'id': 'Arrabbiata-Pasta-1686610',
 'imageUrlsBySize': {'90': 'https://lh3.googleusercontent.com/9JMi7wWx77CVzo4O6OkL4f0qfiLuW3BpJJUXNv00prOJ7zsUvMvm5oLubd16jVkcYWp5Z1hQ7MHohCB3iw2N6g=s90-c'},
 'ingredients': ['tomato sauce',
  'garlic',
  'red pepper flakes',
  'olive oil',
  'dried parsley',
  'salt',
  'dried pasta'],
 'rating': 4,
 'recipeName': 'Arrabbiata Pasta',
 'smallImageUrls': ['https://lh3.googleusercontent.com/BOKHVGhSRJifGx2YwtXs_1HQ6xE_ue3-BNB0we3VQl0DoPs5Xa1rUhOWdHjXQpM1XRL_XnsYd7JBq7Ye2yIp=s90'],
 'sourceDisplayName': 'Three Olives Branch',
 'totalTimeInSeconds': 900}

In [234]:
temp_df = pd.DataFrame(columns=['course','ingredients','recipe_name','rating','cuisine','flavors'])

In [237]:
temp_dict = data1_b['matches'][1]
#temp_df = pd.DataFrame.from_dict(temp_dict)
#pd.DataFrame.from_dict(temp_dict, orient='index').T.set_index('index')
cuisine='italian'
temp_list = []
temp_dict2 = {'course':temp_dict['attributes']['course'], 
              'ingredients':str(temp_dict['ingredients']), 
              'recipe_name':temp_dict['recipeName'], 
              'rating':temp_dict['rating'],
              'cuisine':cuisine}
if temp_dict['flavors'] != None:
    for key,value in temp_dict['flavors'].items():
        temp_dict2[key] = value

    
#temp_df.loc[0,'ingredients']=temp_dict['ingredients']

In [238]:
temp_df = temp_df.append(temp_dict2, ignore_index=True)
temp_df

,course,ingredients,recipe_name,rating,cuisine,flavors,bitter,meaty,piquant,salty,sour,sweet
0,[Main Dishes],"['smoked sausage', 'onions', 'garlic', 'sliced...",One Pot Penne Pasta,4,italian,NaN,0.833333,0.833333,0.333333,0.833333,0.500000,0.166667
1,[Main Dishes],"['whole wheat pasta', 'pasta sauce', 'heavy cr...",Creamy Caprese Pasta,5,italian,NaN,0.166667,0.166667,0.000000,0.333333,0.166667,0.166667


In [353]:
def get_recipes(cuisine='None',num_pages=2):
    '''
    Function to search for recipes from the Yummly API.
    
    Args:
        cuisine (str): Defaults to None.  The list of possible
            cuisines can be found at https://developer.yummly.com/documentation
        num_pages (int): Number of request pages to return.
            If num_pages = 2, will return 20 recipes
            
    Returns: 
        Pandas DataFrame: Each row of the DataFrame will be
            a different recipe.
    '''
    yummly_url = 'http://api.yummly.com/v1/api/recipes?_app_id=ab3c96d5&_app_key=57476e58107ffca18d1585f2b463d78e&your _search_parameters'
    allowedCuisine = 'cuisine^cuisine-{}'.format(cuisine)
    maxResult = 10
    params = {'allowedCuisine[]' :allowedCuisine, 'maxResult' : maxResult, 'start' : 0}
    #print(params)
    yummly_df = pd.DataFrame(columns=['course','ingredients','recipe_name','rating','cuisine'])
    
    for page in range(num_pages):
        params['start']=page*maxResult
        try:
            resp = requests.get(yummly_url, params=params)
        except:
            print('Exception raised when requesting cuisine: {}'.format(cuisine))
            return yummly_df
        #print(resp.status_code)
        #if resp.raise_for_status() == None:
        data = resp.json()
        for i in range(len(data['matches'])):
            yummly_dict = data['matches'][i]
            yummly_dict2 = {'ingredients':str(yummly_dict['ingredients']), 
              'recipe_name':yummly_dict['recipeName'], 
              'rating':yummly_dict['rating'],
              'cuisine':cuisine}
            if yummly_dict['flavors'] != None:
                for key,value in yummly_dict['flavors'].items():
                    yummly_dict2[key] = value
            if 'course' in yummly_dict['attributes'].keys():
                yummly_dict2['course'] = yummly_dict['attributes']['course']
            yummly_df = yummly_df.append(yummly_dict2, ignore_index=True)
        #else:
        #    print('Exception raised when requesting cuisine: {}'.format(cuisine))
    return yummly_df  

In [298]:
#yummly_df_italian = get_recipes(cuisine='italian',num_pages=2)
yummly_df_italian.loc[0,'ingredients']

"['globe eggplant', 'salt', 'olive oil', 'italian seasoning', 'basil leaves', 'freshly grated parmesan', 'mozzarella cheese', 'hot red pepper flakes', 'virgin olive oil', 'large garlic cloves', 'petite diced tomatoes', 'dried oregano']"

In [262]:
#yummly_df_cuban = get_recipes(cuisine='cuban',num_pages=2)
#yummly_df_cuban

In [128]:
yummly_url = 'http://api.yummly.com/v1/api/recipes?_app_id=ab3c96d5&_app_key=57476e58107ffca18d1585f2b463d78e&your _search_parameters'
params = {'allowedCuisine[]' :'cuisine^cuisine-italian', 'maxResult' : 10, 'start' : 0}
resp = requests.get(yummly_url, params=params)

In [129]:
data = resp.json()

In [130]:
data['matches'][0]

{'attributes': {'course': ['Main Dishes', 'Lunch'], 'cuisine': ['Italian']},
 'flavors': {'bitter': 0.8333333333333334,
  'meaty': 0.8333333333333334,
  'piquant': 0.8333333333333334,
  'salty': 0.8333333333333334,
  'sour': 0.8333333333333334,
  'sweet': 0.16666666666666666},
 'id': 'Julia-Childs-Eggplant-Pizzas-2211869',
 'imageUrlsBySize': {'90': 'https://lh3.googleusercontent.com/oiQR3Fo7vLWPJlNTvsJbGzeFSHyMrxeZI_mY9c8s6wngaXbc7Hwlbpw8Pggb9jqyEmg_6fVpvjNpOEI-UbQGlw=s90-c'},
 'ingredients': ['globe eggplant',
  'salt',
  'olive oil',
  'italian seasoning',
  'basil leaves',
  'freshly grated parmesan',
  'mozzarella cheese',
  'hot red pepper flakes',
  'virgin olive oil',
  'large garlic cloves',
  'petite diced tomatoes',
  'dried oregano'],
 'rating': 4,
 'recipeName': 'Julia Child’s Eggplant Pizzas',
 'smallImageUrls': ['https://lh3.googleusercontent.com/6LYPkoWZBy-1TGl9k4NyVpj_mijJCOEKEXq7zIlMrxuqvzNeyBjWXrPuULd3SR3hdVweJVJQsUbdhlV_S6AoXQ=s90'],
 'sourceDisplayName': "Kalyn's K

In [349]:
params = {'allowedCuisine[]' : 'cuisine^cuisine-italian','maxResult' : 10, 'start' : 1000}
resp = requests.get(yummly_url1, params=params)

In [352]:
data2 = resp.json()
#data2.keys()
#data2['totalMatchCount']
data2['matches']

[{'attributes': {'course': ['Main Dishes'],
   'cuisine': ['Kid-Friendly', 'Italian']},
  'flavors': {'bitter': 0.6666666666666666,
   'meaty': 0.16666666666666666,
   'piquant': 0.0,
   'salty': 0.8333333333333334,
   'sour': 0.16666666666666666,
   'sweet': 0.3333333333333333},
  'id': 'Skillet-Chicken-Parmesan-Over-Pasta-1091043',
  'imageUrlsBySize': {'90': 'http://lh3.googleusercontent.com/cIqw8GlLoYjPZ-i6m7Ki1g3fY2fkdELe23Y1BQkP54Wzs-GX1SrZTsplkexey-R48XSJXKLqP72RjYTpEUmlZCs=s90-c'},
  'ingredients': ['chicken breasts',
   'kosher salt',
   'black pepper',
   'Italian herbs',
   'bread crumbs',
   'grated parmesan cheese',
   'butter',
   'olive oil',
   'spaghetti',
   'marinara sauce',
   'shredded mozzarella cheese',
   'fresh herbs'],
  'rating': 4,
  'recipeName': 'Skillet Chicken Parmesan Over Pasta',
  'smallImageUrls': ['http://lh3.googleusercontent.com/8bDFiNowrCeaJXZKOme3nUbOt3ohD_tz1IY3fJZa4eL4tm4LnYCBTsHLvjVR9iSNsUMDeFSgU0GodCLV0lo5-4g=s90'],
  'sourceDisplayName': 'J

In [326]:
len(data2['matches'])

10

In [68]:
data2['matches'][0]

{'attributes': {'course': ['Main Dishes'], 'cuisine': ['Italian']},
 'flavors': {'bitter': 0.5,
  'meaty': 0.16666666666666666,
  'piquant': 0.0,
  'salty': 0.5,
  'sour': 0.16666666666666666,
  'sweet': 0.16666666666666666},
 'id': 'Sweet-potato-cream-pasta-with-crispy-kale-333548',
 'imageUrlsBySize': {'90': 'http://lh5.ggpht.com/xZOFoRq52cIC5ib-W9upZXrWzE6dWjYwIRiaUs7JvDFtwlF9Fx60IkELSiENkjQJAaBSzlDiMpy-iL9SYk357g=s90-c'},
 'ingredients': ['sweet potatoes',
  'olive oil',
  'unsalted butter',
  'shallots',
  'garlic cloves',
  'flour',
  'skim milk',
  'grated romano cheese',
  'whole wheat pasta',
  'kale leaves',
  'toasted sesame oil',
  'salt',
  'pepper'],
 'rating': 5,
 'recipeName': 'Sweet Potato Cream Pasta with Crispy Kale',
 'smallImageUrls': ['http://lh4.ggpht.com/MxUBS-3e2t_eBynY4VavzZIyCDQlBKsB8gYDwn26NLXVVcN5vyDQbVqcrHwONMK8JAMfzfVViuJgQS4kn470vg=s90'],
 'sourceDisplayName': 'How Sweet It Is',
 'totalTimeInSeconds': 3600}

In [354]:
def merge_cuisines(cuisines_list=None, num_pages=2):
    
    merged_df = pd.DataFrame(columns=['course','ingredients','recipe_name','rating','cuisine'])
    
    for cuisine in cuisines_list:
        print("loading in data for cuisine: {}".format(cuisine))
        df = get_recipes(cuisine=cuisine, num_pages=num_pages)
        merged_df = merged_df.append(df, ignore_index=True)
        
    return merged_df

In [277]:
S = 'American, Italian, Asian, Mexican, Southern & Soul Food, French, Southwestern, Barbecue, Indian, Chinese, Cajun & Creole, English, Mediterranean, Greek, Spanish, German, Thai, Moroccan, Irish, Japanese, Cuban, Hawaiian, Swedish, Hungarian, Portugese'
S_list = S.split(',')

In [278]:
S_list_new = []
for S_i in S_list:
    S_i_new = S_i.split('&')[0].lower().strip()
    S_list_new.append(S_i_new)

In [279]:
A = 'American'
A.split('&')[0].lower()

'american'

In [280]:
S_list_new

['american',
 'italian',
 'asian',
 'mexican',
 'southern',
 'french',
 'southwestern',
 'barbecue',
 'indian',
 'chinese',
 'cajun',
 'english',
 'mediterranean',
 'greek',
 'spanish',
 'german',
 'thai',
 'moroccan',
 'irish',
 'japanese',
 'cuban',
 'hawaiian',
 'swedish',
 'hungarian',
 'portugese']

In [328]:
yummly_df_merged = merge_cuisines(S_list_new,num_pages=10)

loading in data for cuisine: american
loading in data for cuisine: italian
loading in data for cuisine: asian
loading in data for cuisine: mexican
loading in data for cuisine: southern
loading in data for cuisine: french
loading in data for cuisine: southwestern
loading in data for cuisine: barbecue
loading in data for cuisine: indian
loading in data for cuisine: chinese
loading in data for cuisine: cajun
loading in data for cuisine: english
loading in data for cuisine: mediterranean
Exception raised when requesting cuisine: mediterranean
loading in data for cuisine: greek
loading in data for cuisine: spanish
loading in data for cuisine: german
loading in data for cuisine: thai
loading in data for cuisine: moroccan
loading in data for cuisine: irish
loading in data for cuisine: japanese
Exception raised when requesting cuisine: japanese
loading in data for cuisine: cuban
Exception raised when requesting cuisine: cuban
loading in data for cuisine: hawaiian
Exception raised when requesti

In [338]:
sum(yummly_df_merged['cuisine']=='mediterranean')

60

In [333]:
df_temp = get_recipes(cuisine='cuban',num_pages=2)

In [334]:
df_temp

,course,ingredients,recipe_name,rating,cuisine,bitter,meaty,piquant,salty,sour,sweet
0,"[Beverages, Cocktails]","['lime', 'mint leaves', 'sugar', 'white rum', ...",White Christmas Mojito.,4,cuban,NaN,NaN,NaN,NaN,NaN,NaN
1,[Lunch],"['buns', 'mustard', 'swiss cheese', 'pork loin...",Cuban Sandwich,5,cuban,NaN,NaN,NaN,NaN,NaN,NaN
2,[Side Dishes],"['olive oil', 'onions', 'garlic', 'scallions',...",Cuban Style Black Beans,4,cuban,0.166667,0.166667,0.166667,0.833333,0.500000,0.166667
3,[Main Dishes],"['cumin', 'garlic', 'red chili peppers', 'pepp...",Cuban Chicken and Black Bean Quinoa Bowls with...,4,cuban,0.166667,0.166667,0.833333,0.166667,0.833333,0.333333
4,"[Beverages, Cocktails]","['mint leaves', 'sugar', 'lime juice', 'lime',...",The Ultimate Mojito,5,cuban,NaN,NaN,NaN,NaN,NaN,NaN
5,[Lunch],"['bread', 'ham', 'pork', 'swiss cheese', 'dill...",Traditional Cuban Sandwich (Cubano),4,cuban,NaN,NaN,NaN,NaN,NaN,NaN
6,[Main Dishes],"['bone in', 'garlic', 'onions', 'Seville orang...",Cuban Roast Pork (Lechon Asado),5,cuban,NaN,NaN,NaN,NaN,NaN,NaN
7,[Side Dishes],"['corn husks', 'melted butter', 'grated cotija...",Cuban Grilled Corn,5,cuban,NaN,NaN,NaN,NaN,NaN,NaN
8,[Lunch],"['boiled ham', 'pork loin roast', 'swiss chees...",Cubano Sandwich,4,cuban,NaN,NaN,NaN,NaN,NaN,NaN
9,"[Beverages, Cocktails]","['sugar', 'mint leaves', 'lime', 'light rum', ...",Red Mojitos,5,cuban,NaN,NaN,NaN,NaN,NaN,NaN


In [356]:
yummly_df_merged_large = merge_cuisines(S_list_new,num_pages=20)

loading in data for cuisine: american
loading in data for cuisine: italian
loading in data for cuisine: asian
Exception raised when requesting cuisine: asian
loading in data for cuisine: mexican
loading in data for cuisine: southern
loading in data for cuisine: french
Exception raised when requesting cuisine: french
loading in data for cuisine: southwestern
loading in data for cuisine: barbecue
loading in data for cuisine: indian
loading in data for cuisine: chinese
loading in data for cuisine: cajun
loading in data for cuisine: english
loading in data for cuisine: mediterranean
loading in data for cuisine: greek
loading in data for cuisine: spanish
Exception raised when requesting cuisine: spanish
loading in data for cuisine: german
Exception raised when requesting cuisine: german
loading in data for cuisine: thai
Exception raised when requesting cuisine: thai
loading in data for cuisine: moroccan
loading in data for cuisine: irish
Exception raised when requesting cuisine: irish
loadi

In [370]:
sum(yummly_df_merged_large['cuisine']=='swedish')

20

In [371]:
S_list_new[-3]

'swedish'

In [379]:
yummly_df_swedish = get_recipes(cuisine='swedish',num_pages=100)

In [380]:
yummly_df_swedish.shape

(929, 11)

In [381]:
yummly_df_merged_last3 = merge_cuisines(S_list_new[-3:],num_pages=20)

loading in data for cuisine: swedish
loading in data for cuisine: hungarian
loading in data for cuisine: portugese


In [382]:
yummly_df_merged_last3.shape

(599, 11)

In [383]:
yummly_df_merged_first5 = merge_cuisines(S_list_new[:5],num_pages=50)

loading in data for cuisine: american
loading in data for cuisine: italian
loading in data for cuisine: asian
loading in data for cuisine: mexican
loading in data for cuisine: southern


In [384]:
yummly_df_merged_first5.shape

(2500, 11)

In [386]:
# %load 383
yummly_df_merged_second5 = merge_cuisines(S_list_new[5:10],num_pages=50)

loading in data for cuisine: french
loading in data for cuisine: southwestern
loading in data for cuisine: barbecue
loading in data for cuisine: indian
loading in data for cuisine: chinese


In [388]:
yummly_df_merged_third5 = merge_cuisines(S_list_new[10:15],num_pages=50)

loading in data for cuisine: cajun
loading in data for cuisine: english
loading in data for cuisine: mediterranean
loading in data for cuisine: greek
loading in data for cuisine: spanish


In [390]:
# %load 388
yummly_df_merged_fouth5 = merge_cuisines(S_list_new[15:20],num_pages=50)

loading in data for cuisine: german
loading in data for cuisine: thai
loading in data for cuisine: moroccan
loading in data for cuisine: irish
loading in data for cuisine: japanese


In [392]:
np.shape(S_list_new)

(25,)

In [394]:
yummly_df_merged_fifth5 = merge_cuisines(S_list_new[20:25],num_pages=50)

loading in data for cuisine: cuban
loading in data for cuisine: hawaiian
loading in data for cuisine: swedish
loading in data for cuisine: hungarian
loading in data for cuisine: portugese


In [409]:
yummly_df_merged_large2 = pd.concat([yummly_df_merged_first5,yummly_df_merged_second5,
                                    yummly_df_merged_third5,yummly_df_merged_fouth5,
                                    yummly_df_merged_fifth5], ignore_index=True)

In [410]:
yummly_df_merged_large2.shape

(12492, 11)

In [411]:
yummly_df_merged_large2.to_pickle('yummly_df.pkl')

In [412]:
yummly_df = pd.read_pickle('yummly_df.pkl')

In [413]:
yummly_df.head()

,bitter,course,cuisine,ingredients,meaty,piquant,rating,recipe_name,salty,sour,sweet
0,0.666667,[Main Dishes],american,"['dried pasta', 'milk', 'shredded cheddar chee...",0.166667,0.166667,4,Revolutionary Mac & Cheese,0.833333,0.166667,0.166667
1,0.500000,[Salads],american,"['tomatoes', 'avocado', 'red onion', 'chopped ...",0.166667,0.000000,4,Avocado and Tomato Salad,0.166667,0.833333,0.166667
2,NaN,"[Breakfast and Brunch, Breads]",american,"['melted butter', 'biscuit dough', 'fresh mozz...",NaN,NaN,5,Easy Cheesy Bacon Biscuit Pull-Aparts,NaN,NaN,NaN
3,NaN,[Side Dishes],american,"['cauliflower', 'extra-virgin olive oil', 'red...",NaN,NaN,5,Roasted Spicy Cauliflower,NaN,NaN,NaN
4,0.833333,NaN,american,"['yukon gold potatoes', 'salt', 'smoked paprik...",0.166667,0.166667,5,Shakin’ Hash Browns,0.166667,0.666667,0.000000


In [414]:
yummly_df.tail()

,bitter,course,cuisine,ingredients,meaty,piquant,rating,recipe_name,salty,sour,sweet
12487,0.333333,[Salads],portugese,"['cherry tomatoes', 'fresh mozzarella cheese',...",0.166667,0.0,4,Caprese Salad Skewers,0.166667,0.666667,0.333333
12488,NaN,NaN,portugese,"['pie crust', 'large egg yolks', 'unsalted mar...",NaN,NaN,4,Perfect Pecan Pie,NaN,NaN,NaN
12489,NaN,[Breads],portugese,"['warm water', 'sugar', 'active dry yeast', 'g...",NaN,NaN,4,Super Easy Gluten-Free Naan Bread,NaN,NaN,NaN
12490,NaN,[Appetizers],portugese,"['chicken wings', 'dijon mustard', 'honey', 'r...",NaN,NaN,4,5-Ingredient Slow Cooker Honey-Dijon Chicken W...,NaN,NaN,NaN
12491,0.166667,[Condiments and Sauces],portugese,"['white wine', 'cream', 'butter', 'fresh lemon...",0.166667,0.0,4,Lemon Butter Sauce,0.166667,0.333333,0.166667


Let's look at our class balance.

In [415]:
yummly_df.groupby('cuisine').count()

,bitter,course,ingredients,meaty,piquant,rating,recipe_name,salty,sour,sweet
cuisine,,,,,,,,,,
american,315,467,500,315,315,500,500,315,315,315
asian,336,433,500,336,336,500,500,336,336,336
barbecue,336,449,500,336,336,500,500,336,336,336
cajun,380,443,500,380,380,500,500,380,380,380
chinese,364,437,500,364,364,500,500,364,364,364
cuban,273,465,500,273,273,500,500,273,273,273
english,336,481,500,336,336,500,500,336,336,336
french,368,470,500,368,368,500,500,368,368,368
german,370,401,500,370,370,500,500,370,370,370


The above shows that we are evenly balanced with approximately 500 recipes for each cuisine type.

In [416]:
yummly_df['ingredients'][1]

"['tomatoes', 'avocado', 'red onion', 'chopped cilantro', 'lime', 'extra-virgin olive oil', 'salt']"